In [1]:
### Simulation of an arrival process wiU arrival rate lambda(t), 0<t<T
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from tqdm import tqdm
import pickle
import torch
from torch.distributions.exponential import Exponential
import pandas as pd
import random
import seaborn

In [2]:
def pareto_minus_1_cdf(x, a):
    return 1-(x+1)**(-a)

def inverse_pareto_minus_1_cdf(x, a):
    return (1-x)**(-1/a)-1

def sample_symmetric_pareto(a, shape):
    uniform_samples = np.random.uniform(0,1,shape)
    pareto_samples = inverse_pareto_minus_1_cdf(uniform_samples, a)
    bernoulli_mult = np.random.choice([-1, 1], size=shape)
    symmetric_pareto_samples = pareto_samples*bernoulli_mult
    return symmetric_pareto_samples

In [3]:
def VAR_pareto(phi_matrix_list, alpha=2, size = 10000, burn = 1000):
    # Generate stationary time series data
    p = len(phi_matrix_list)
    y = np.zeros((size+burn, p))
    
    # Simulate the VAR process
    for t in range(p, size+burn):
        for dim in range(p):
            y[t] = y[t] + phi_matrix_list[dim] @ y[t-1-dim]
        noise = sample_symmetric_pareto(alpha, (p))
        y[t] = y[t] + noise
    return y[-size:]


def VAR_pareto_cond(conditions, phi_matrix_list, alpha=2, obs = 10000, sample_total_len = 10):
    # Generate stationary time series data
    p = len(phi_matrix_list)
    assert p==conditions.shape[1]
    y = np.zeros((obs, sample_total_len, p))
    copied_conditions = np.repeat(conditions[np.newaxis, :, :], obs, axis=0)
    y[:,:len(conditions),:] = copied_conditions

    # Simulate the VAR process
    for i in range(obs):
        for t in range(conditions.shape[0], sample_total_len):
            for dim in range(p):
                if t-1-dim<0:
                    break
                y[i,t,:] = y[i,t,:] + phi_matrix_list[dim] @ y[i,t-1-dim,:]
            noise = sample_symmetric_pareto(alpha, (p))
            y[i,t,:] = y[i,t,:] + noise
    return y


In [4]:
# matrix
A1 = np.array([[0.7, -0.1, 0.3],
               [-0.1, 0.4, 0.1],
               [0.3, 0.1, -0.8]])

A2 = np.array([[0.1, 0.05, 0.05],
               [0.05, -0.4, 0.05],
               [0.05, 0.05, 0.1]])

A3 = np.array([[0.05, 0.02, 0.02],
               [0.02, 0.05, 0.02],
               [0.02, 0.02, 0.05]])

from numpy.linalg import eig
companion_matrix = np.block([
    [A1, A2, A3],
    [np.eye(3), np.zeros((3, 6))],
    [np.zeros((3, 3)), np.eye(3), np.zeros((3, 3))]
])

# Check the eigenvalues of the companion matrix
eigenvalues = eig(companion_matrix)[0]
print("Eigenvalues of the companion matrix:", eigenvalues)

# Ensure the eigenvalues are less than 1 in magnitude
if all(np.abs(eigenvalues) < 1):
    print("The process is stationary.")
else:
    print("The process is not stationary.")

alpha = 3.0

Eigenvalues of the companion matrix: [ 0.94950619+0.j         -0.90078321+0.j          0.13481259+0.57096677j
  0.13481259-0.57096677j -0.08354563+0.2326175j  -0.08354563-0.2326175j
 -0.1926829 +0.j          0.24665492+0.j          0.09477109+0.j        ]
The process is stationary.


In [5]:
# unconditional generation
np.random.seed(1)
num_obs = 100000
sample_length = 10
dim = 3
long_path = VAR_pareto([A1,A2,A3], alpha = alpha, size=num_obs+sample_length, burn=1000)
# split into samples:
unconditional_samples = np.zeros((num_obs,sample_length,dim))
for i in range(num_obs):
    unconditional_samples[i] = long_path[i:i+sample_length,:]

In [6]:
# conditional generation
np.random.seed(1)
conditional_dim = 5
conditions = VAR_pareto([A1,A2,A3], alpha = alpha, size=conditional_dim, burn=2000)
conditional_samples = VAR_pareto_cond(conditions, [A1,A2,A3], alpha = alpha, obs=10000, sample_total_len=10)

In [7]:
np.save("var_pareto_unconditional.npy", unconditional_samples)
np.save("var_pareto_conditional.npy", conditional_samples)